In [ ]:
## naver api를 바탕으로 뉴스 크롤링
## 크롤링한 기사의 본문을 크롤링하여 chat-GPT를 활용해 가짜 뉴스 생성

In [21]:
import os
import sys
import urllib.request
import urllib.parse
import re
import json

# sid 추출을 위한 카테고리
sid2category = {
    "100" : "정치",
    "101" :"경제",
    "102" : "사회",
    "103" :"생활/문화",
    "104" : "세계",
    "105" : "IT/과학",
}

# 네이버 인증정보(네이버 api에서 넣기)
client_id = "YOUR_NAVER_DEVELOP_ID"
client_secret = "YOUR_NAVER_DEVELOP_SECRET"

# 검색어 설정
query = "트럼프"
display = 10
start = 1
encText = urllib.parse.quote(query)

# 네이버 뉴스 api url
url = "https://openapi.naver.com/v1/search/news.json?query=%s&display=%d&start=%d&sort=sim" % (encText, display, start)
param = "?query=%s&display=10&start=1&sort=sim" # JSON 결과

# api 요청 설정
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

# api 호출
response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    data = json.loads(response_body.decode('utf-8')) #JSON 객체로 변환 -> get 사용을 위해

    # 뉴스 제목과 링크 추출
    news_list = []
    for item in data.get("items", []):
      title = item.get("title", "").replace("<b>", "").replace("</b>", "").replace("&quot;", "\"") # 태그제거
      link = item.get("link", "")

      # sid 추출
      match = re.search(r"sid=(\d+)", link)
      sid = match.group(1) if match else "기타"

      # 카테고리 매핑
      category = sid2category.get(sid, "기타")

      # 저장
      news_list.append({
          "title": title,
          "link": link,
          "category" : category})
    print(news_list)

else:
    print("Error Code:" + rescode)

HTTPError: HTTP Error 401: Unauthorized

In [18]:
# 위의 함수에서 얻은 link를 가지고 본문을 크롤링하는 함수
import requests
from bs4 import BeautifulSoup
import time

def get_naver_news(news_url):
  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
  }

  try:
    response = requests.get(news_url, headers=headers)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, "html.parser")
      article_body = soup.select_one("#dic_area")

      return article_body.get_text(strip=True) if article_body else "기사 본문을 찾을 수 없습니다."

    else:
      return f"에러 코드 : {response.status_code}"

  except Exception as e:
    return f"오류 발생 : {e}"


## 본문 크롤링하고 저장할 리스트
url2article = []

for news in news_list:
  content = get_naver_news(news["link"])
  news["content"] = content
  news["use_type"] = 1

  url2article.append(news)
  time.sleep(1) # 서버 부하 방지

## 잘 작동됨

위에서 만든 리스트를 json 파일로 저장

In [ ]:
with open("real_news.json", "w", encoding="utf-8") as f:
  json.dump(url2article, f, ensure_ascii=False, indent = 4)